In [36]:
# Append web folder to system path
import sys
sys.path.append('../web')

In [37]:
from db_config import Base
from Database import Station, Availability, Weather
from sqlalchemy import create_engine, func, Column, String, Integer, Double, Boolean
from sqlalchemy.orm import sessionmaker
import json
import sys
import requests
from datetime import datetime, timedelta

In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

In [39]:
# Get the db_info
with open('../dbinfo.json') as f:
    db_info = json.load(f)


USER = db_info['dbConnection']['USER']
PASSWORD = db_info['dbConnection']['PASSWORD']
URI = db_info['dbConnection']['URI']
PORT = db_info['dbConnection']['PORT']
DB = db_info['dbConnection']['DB']
WEATHER_API_KEY = db_info['weatherKey']


# Create a new session
engine = create_engine(
    'mysql+pymysql://{}:{}@localhost:{}/{}'.format(USER, PASSWORD, PORT, DB), echo=True)
Base.metadata.create_all(bind=engine)
Session = sessionmaker(bind=engine)
session = Session()
print("connected")

2024-03-31 11:10:24,162 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-03-31 11:10:24,165 INFO sqlalchemy.engine.Engine [raw sql] {}


2024-03-31 11:10:24,197 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-03-31 11:10:24,197 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-31 11:10:24,213 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-03-31 11:10:24,214 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-31 11:10:24,274 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-31 11:10:24,291 INFO sqlalchemy.engine.Engine DESCRIBE `db_on_your_bike`.`stations`
2024-03-31 11:10:24,292 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-31 11:10:24,528 INFO sqlalchemy.engine.Engine DESCRIBE `db_on_your_bike`.`availability`
2024-03-31 11:10:24,530 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-31 11:10:24,547 INFO sqlalchemy.engine.Engine DESCRIBE `db_on_your_bike`.`weather`
2024-03-31 11:10:24,548 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-31 11:10:24,779 INFO sqlalchemy.engine.Engine COMMIT
connected


In [40]:
midnight = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

weather_historical = session.query(Weather).filter(
        Weather.time_updated > midnight).all()

# for row in weather_historical:
#     print(row.time_updated, row.wind_speed, row.temperature, row.humidity)

weather_historical_df = pd.DataFrame([row.__dict__ for row in weather_historical])
weather_historical_df = weather_historical_df[['time_updated', 'temperature', 'wind_speed', 'humidity']]
weather_historical_df

2024-03-31 11:10:24,824 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-31 11:10:24,832 INFO sqlalchemy.engine.Engine SELECT weather.time_updated AS weather_time_updated, weather.type AS weather_type, weather.description AS weather_description, weather.temperature AS weather_temperature, weather.feels_like AS weather_feels_like, weather.min_temp AS weather_min_temp, weather.max_temp AS weather_max_temp, weather.humidity AS weather_humidity, weather.wind_speed AS weather_wind_speed, weather.visibility AS weather_visibility, weather.clouds AS weather_clouds, weather.sunrise AS weather_sunrise, weather.sunset AS weather_sunset 
FROM weather 
WHERE weather.time_updated > %(time_updated_1)s
2024-03-31 11:10:24,833 INFO sqlalchemy.engine.Engine [generated in 0.00105s] {'time_updated_1': datetime.datetime(2024, 3, 31, 0, 0)}


,time_updated,temperature,wind_speed,humidity
0,2024-03-31 00:00:05,8.42,4.12,88
1,2024-03-31 00:30:06,8.38,5.66,85
2,2024-03-31 01:00:05,7.95,5.66,84
3,2024-03-31 01:30:05,8.11,4.63,85
4,2024-03-31 02:00:04,8.19,5.66,86
5,2024-03-31 02:30:05,7.90,5.14,90
6,2024-03-31 03:00:05,7.79,5.14,91
7,2024-03-31 03:30:05,7.78,5.66,92
8,2024-03-31 04:00:05,7.82,5.66,92
9,2024-03-31 04:30:05,8.24,5.66,91


In [41]:
# Weather
# Weather URI
WEATHER_URI = 'https://api.openweathermap.org/data/2.5/forecast'
weather_data = requests.get(WEATHER_URI, params={
                            "units": "metric", "lat": 53.344, "lon": -6.2672, "appid": WEATHER_API_KEY})
weather_info = weather_data.json()

In [42]:
station_id = 2

# use pd.DataFrame because data is already an object
predicted_weather_df = pd.DataFrame(weather_info['list'])
predicted_weather_df['temperature'] = [row['main']['temp'] for row in weather_info['list']]
predicted_weather_df['humidity'] = [row['main']['humidity'] for row in weather_info['list']]
predicted_weather_df['wind_speed'] = [row['wind']['speed'] for row in weather_info['list']]
predicted_weather_df['time_updated'] = pd.to_datetime(predicted_weather_df['dt_txt'])

predicted_weather_df['humidity'] = predicted_weather_df['humidity'].astype('int64')

# Convert temperature and wind_speed to float64
predicted_weather_df['temperature'] = predicted_weather_df['temperature'].astype('float64')
predicted_weather_df['wind_speed'] = predicted_weather_df['wind_speed'].astype('float64')


predicted_weather_df = predicted_weather_df[['time_updated', 'temperature', 'wind_speed', 'humidity']]

predicted_weather_df



,time_updated,temperature,wind_speed,humidity
0,2024-03-31 12:00:00,10.74,5.84,80
1,2024-03-31 15:00:00,10.87,5.58,76
2,2024-03-31 18:00:00,9.08,5.51,82
3,2024-03-31 21:00:00,8.61,6.76,88
4,2024-04-01 00:00:00,7.72,6.61,91
5,2024-04-01 03:00:00,7.33,5.76,95
6,2024-04-01 06:00:00,7.64,3.78,98
7,2024-04-01 09:00:00,8.65,2.20,85
8,2024-04-01 12:00:00,11.48,3.52,66
9,2024-04-01 15:00:00,10.78,4.54,67


In [43]:
current_date = datetime.now()

# Generate a list of hours for today
hours_today = [current_date.replace(hour=h, minute=0, second=0, microsecond=0) for h in range(24)]


In [44]:
weather_combined = pd.concat([weather_historical_df, predicted_weather_df])
weather_combined.head()

,time_updated,temperature,wind_speed,humidity
0,2024-03-31 00:00:05,8.42,4.12,88
1,2024-03-31 00:30:06,8.38,5.66,85
2,2024-03-31 01:00:05,7.95,5.66,84
3,2024-03-31 01:30:05,8.11,4.63,85
4,2024-03-31 02:00:04,8.19,5.66,86


In [45]:
days = ['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
       'Wednesday']

# One hot encode day of the week
weather_combined['weekday'] = weather_combined['time_updated'].dt.day_name()
weather_combined['hour'] = weather_combined['time_updated'].dt.hour
for day in days:
    weather_combined[day] = weather_combined['weekday'] == day

weather_combined.drop('time_updated', axis = 1, inplace=True)
weather_combined.drop('weekday', axis = 1, inplace=True)

with open(f'station_{station_id}.pkl', 'rb') as file:
    # Load the model from the file
    poly_reg_model = pickle.load(file)

poly = PolynomialFeatures(degree=3, include_bias=False)
poly_features = poly.fit_transform(predicted_weather_df)


weather_combined['predicted_available'] = poly_reg_model.predict(poly_features)

DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Float64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Int64DType'>)

In [ ]:
predicted_weather_df.head()

,temperature,wind_speed,humidity,hour,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,predicted_bike_stands
0,12.13,3.37,72,18,True,False,False,False,False,False,False,-0.457117
1,10.01,2.91,80,21,True,False,False,False,False,False,False,-0.933004
2,7.48,2.20,88,0,False,False,True,False,False,False,False,-5.925614
3,3.37,1.91,92,3,False,False,True,False,False,False,False,6.688182
4,3.37,2.18,91,6,False,False,True,False,False,False,False,10.187085
